![DT Standalone + EUMDAC](./img/DataTailor_EUMETSAT.png)

Copyright (c) 2024 EUMETSAT  
License: MIT

<hr>

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_2_Cleaning_the_Data_Tailor_workspace.ipynb">← Cleaning the Data Tailor workspace</a><span style="float:right;"><a href="./4_EUMDAC-Cookbook.ipynb">EUMDAC Cookbook →</a>

## Using the EUMETSAT Data Tailor Standalone with EUMDAC

#### What will this module teach you?

This module will show you how to:
1. Select a range products from the **Data Store** via EUMDAC library
2. Pass the downloaded product to the **Data Tailor Standalone**
3. Retrieve your customised products

#### Prerequisites:
For this workflow you will need a conda environment with below software installed.
1. EUMETSAT Data Access Client (https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide)
2. The Data Tailor Standalone (https://user.eumetsat.int/resources/user-guides/data-tailor-standalone-guide)
3. Jupyter Notebook (https://anaconda.org/conda-forge/jupyter/)

**Note:** You should run this notebook in the same environment as you have your local Data Tailor instance installed.

### Use EUMDAC library

In this section we will demonstrate how you can use the **EUMETSAT Data Access Client (EUMDAC)** to download products you select from the Data Store. The products will be downloaded in their **native/default format**.

#### Authentication

After installing the EUMDAC we are calling the **token**. It takes care of requesting a new value after expiration.

In [ ]:
# Import the library EUMDAC
import eumdac

# Import the DT Standalone library
from epct import api

# Import other necassery libraries
import time
import requests
import datetime
import shutil
import os
import glob

In [ ]:
# Insert your personal key and secret into the single quotes

consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

In [ ]:
try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when trying the request to the server: '{exc}'")

### Selecting and downloading a range of products from the Data Store

For this demonstration, we are going to select a range of Level 1.5 SEVIRI products (collection ID: EO:EUM:DAT:MSG:HRSEVIRI) and download those. This sensor is aboard the geostationary MSG platform.

In [ ]:
# Feed the eumdac datastore module with API token
datastore = eumdac.DataStore(token)

# Select the collection
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')

# Define data/time search filters
start = datetime.datetime(2022, 9, 22, 19, 20)
end = datetime.datetime(2022, 9, 22, 19, 50)

# Retrieve datasets that match our filter
products = selected_collection.search(
    dtstart=start, 
    dtend=end,
    sort="start,time,1")

In [ ]:
# Print found datasets
try:
    print(f'Found {products.total_results} datasets for the given time range:') 
    for product in products:
        print(product)
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

## Downloading & Customising products with EUMDAC + the Data Tailor Standalone

Now that we have the products found, we can go ahead and download & tailor our products. We will use EUMDAC to download products and DT Standalone to customise them inside a single loop. 

Modify your chain configuration and target directory:

In [5]:
# Define a chain configuration for customising the products
chain_config = {
    'product': 'HRSEVIRI',
    'format': 'jpeg_rgb',
    'filter': {"bands" : ["channel_3","channel_2","channel_1"]}
                }

# Define your target folder for downloads (Note: The directory need to exist before you proceed.)
target_dir = "/home/USERNAME/eumdac_data_store/"

<div class="alert alert-block alert-info">
<b>NOTE:</b> Guidance on chain customisation can be found on our <a href="https://user.eumetsat.int/resources/user-guides/data-store-detailed-guide#ID-Understanding-configuring-and-using-chains">Understanding, Configuring and Using Chains</a> page.
</div>

Now, we can run the following code cell to loop through our downloaded products and customising them with our running Data Tailor instance:

In [ ]:
for product in products:

    # Open the source file associated with the product for reading
    with product.open() as source_file:  
        destination_file_name = source_file.name
        # Open (or create) the destination file for writing binary data
        with open(destination_file_name, mode='wb') as destination_file:
            print(f'Download of product {product} started.')
            shutil.copyfileobj(source_file, destination_file)
        print(f'Download of product {product} finished. Customisation is starting.')

        filename = str(product)+'.zip'
        # change to DT standalone
        output_files = api.run_chain(
            product_paths=[filename],
            chain_config=chain_config,
            target_dir=target_dir
        )

        print("Customisation is finished and saved to:",output_files)

        if os.path.exists(filename):
            os.remove(filename)
            print(f"Downloaded product {filename} has been deleted.")
        else:
            print(f"Downloaded product {filename} does not exist.")

    # Pattern match for files ending with .aux.xml
    for filepath in glob.glob(f'{target_dir}*.aux.xml'):
        os.remove(filepath)
        print(f'Removed: {filepath}')

    # Pattern match for files ending with .wld
    for filepath in glob.glob(f'{target_dir}*.wld'):
        os.remove(filepath)
        print(f'Removed: {filepath}')

This ends our example on how to perform an end-to-end workflow on Data Store products, downloading them first and then using the Data Tailor standalone to customise them.

A comprehensive guide on how to use EUMDAC with the Data Tailor Standalone, can be find in our user guide: https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Data-Tailor-Standalone-in-EUMDAC

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_2_Cleaning_the_Data_Tailor_workspace.ipynb">← Cleaning the Data Tailor workspace</a><span style="float:right;"><a href="./4_EUMDAC-Cookbook.ipynb">EUMDAC Cookbook →</a>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>